In [750]:
import pandas as pd
from datetime import datetime, date

In [751]:
df = pd.DataFrame(
    {
        'Date': [], 
        'Name': [],
        'Holding_Period': [],
        'Trade_Type': [],
        'Buy': [],
        'Sell': [],
        'Quantity': [],
        'Profit/Loss': [],
        'Profit/Loss_%': [],
        'Actual_Profit_Loss_%': [],
        'Actual_Profit_Loss_%': [],
    }
)

df.to_csv('trade_record.csv', index=False)
df.to_csv('trade_record_test.csv', index=False)

In [752]:
class TradeRecordKeeperIntraday:

    def __init__(self, stock_name, buy, sell, quantity, holding_period='Intraday', trade_type='Short'):
        self.stock_name = stock_name
        self.buy = buy
        self.sell = sell
        self.quantity = quantity
        self.holding_period = 'Intraday'
        self.trade_type = 'Short'
        self.filename = 'trade_record.csv'
        self.filename_test = 'trade_record_test.csv'

    def get_csv_dataframe(self, filename):
        if filename[-4:] == '.csv':
            df = pd.read_csv(filename)
        else:
            df = pd.read_csv(f"{filename}.csv")

        return df

    def calc_profit_loss(self,):
        return (self.sell - self.buy) * self.quantity

    def calc_profit_loss_percent(self,):
        return self.calc_profit_loss() * 100/(self.buy * self.quantity)

    def calc_brokerage_per_txn(self, price, quantity):
        brokerage = 20 if (0.0003*price*quantity) > 20 else (0.0003*price*quantity)
        return brokerage
            
    def calc_turnover(self, buy, sell, quantity):
        return (buy + sell) * quantity

    def calc_STT_CTT(self, sell, quantity):
        return 0.00025 * sell * quantity

    def calc_txn_charges(self, buy, sell, quantity):
        return 0.0000345 * (buy + sell) * quantity

    def calc_gst(self, total_brokerage, txn_charges):
        return 0.18 * (total_brokerage + txn_charges)

    def calc_sebi_charges_per_txn(self, price, quantity):
        return (10 / 10**7) * price * quantity

    def calc_clearing_charges(self,):
        return 0

    def calc_stamp_charges(self, buy, quantity):
        return (300 / 10**7) * buy * quantity

    def calc_total_tax(self,):

        results = {
            'total_brokerage': round(self.calc_brokerage_per_txn(self.buy, self.quantity) + self.calc_brokerage_per_txn(self.sell, self.quantity), 2),
            'stt': round(self.calc_STT_CTT(self.sell, self.quantity)),
            'txn_charges': round(self.calc_txn_charges(self.buy, self.sell, self.quantity), 2),
            'clearing_charges': round(self.calc_clearing_charges(), 2),
            'total_sebi_charges': round(self.calc_sebi_charges_per_txn(self.buy, self.quantity) + self.calc_sebi_charges_per_txn(self.sell, self.quantity), 2),
            'stamp_duty': round(self.calc_stamp_charges(self.buy, self.quantity), 2)
        }

        results['gst'] = round(self.calc_gst(results['total_brokerage'], results['txn_charges']), 2)

        total_tax_charges = sum(results.values())
        
        return {
            'results': results, 
            'total_tax_charges': total_tax_charges,
        }
    
    def calc_actual_profit_loss(self,):
        profit = self.calc_profit_loss()
        total_tax_calc = self.calc_total_tax()
        actual_profit = profit - total_tax_calc['total_tax_charges']
        actual_profit_percent = actual_profit * 100/(self.buy * self.quantity)
        
        return {
            'actual_profit_loss': actual_profit,
            'actual_profit_loss_percent': actual_profit_percent,
        }
    
    def append_record(self,):
        results, total_tax_charges = self.calc_total_tax()

        record = {
            'Date': date.today(),
            'Name': self.stock_name,
            'Holding_Period': self.holding_period,
            'Trade_Type': self.trade_type,
            'Buy': self.buy,
            'Sell': self.sell,
            'Quantity': self.quantity,
            'Profit/Loss': round(self.calc_profit_loss(), 2),
            'Profit/Loss_%': round(self.calc_profit_loss_percent(), 2),
            'Actual_Profit_Loss': round(self.calc_actual_profit_loss()['actual_profit_loss'], 2),
            'Actual_Profit_Loss_%': round(self.calc_actual_profit_loss()['actual_profit_loss_percent'], 2)
        }

        df = obj.get_csv_dataframe('trade_record')
        df_test = obj.get_csv_dataframe('trade_record_test')

        df.append(record, ignore_index=True).to_csv('trade_record.csv',index=False)
        df_test.append(record, ignore_index=True).to_csv('trade_record_test.csv', mode='a',index=False)

        return record

In [760]:
TradeRecordKeeperIntraday('ASIANPAINTS', 3472.15, 3529.4, 2).append_record()

{'Date': datetime.date(2022, 1, 13),
 'Name': 'ASIANPAINTS',
 'Holding_Period': 'Intraday',
 'Trade_Type': 'Short',
 'Buy': 3472.15,
 'Sell': 3529.4,
 'Quantity': 2,
 'Profit/Loss': 114.5,
 'Profit/Loss_%': 1.65,
 'Actual_Profit_Loss': 106.76,
 'Actual_Profit_Loss_%': 1.54}

In [761]:
df = obj.get_csv_dataframe('trade_record')
df_test = obj.get_csv_dataframe('trade_record_test')

In [762]:
df

,Date,Name,Holding_Period,Trade_Type,Buy,Sell,Quantity,Profit/Loss,Profit/Loss_%,Actual_Profit_Loss_%,Actual_Profit_Loss
0,2022-01-13,DLF,Intraday,Short,410.50,412.1,12,19.2,0.39,0.29,14.15
1,2022-01-13,ASIANPAINTS,Intraday,Short,3472.15,3529.4,2,114.5,1.65,1.54,106.76


In [763]:
df_test

,Date,Name,Holding_Period,Trade_Type,Buy,Sell,Quantity,Profit/Loss,Profit/Loss_%,Actual_Profit_Loss_%
Date,Name,Holding_Period,Trade_Type,Buy,Sell,Quantity,Profit/Loss,Profit/Loss_%,Actual_Profit_Loss_%,Actual_Profit_Loss
2022-01-13,DLF,Intraday,Short,410.5,412.1,12,19.2,0.39,0.29,14.15
Date,Name,Holding_Period,Trade_Type,Buy,Sell,Quantity,Profit/Loss,Profit/Loss_%,Actual_Profit_Loss_%,Actual_Profit_Loss
Name,Holding_Period,Trade_Type,Buy,Sell,Quantity,Profit/Loss,Profit/Loss_%,Actual_Profit_Loss_%,Actual_Profit_Loss,NaN
DLF,Intraday,Short,410.5,412.1,12,19.2,0.39,0.29,14.15,NaN
2022-01-13,ASIANPAINTS,Intraday,Short,3472.15,3529.4,2,114.5,1.65,1.54,106.76
